[![](https://img.youtube.com/vi/FnV0thLS1Fs/0.jpg)](https://www.youtube.com/watch?v=FnV0thLS1Fs)

pixyz  
last update 2022 05 08  
ゆっくりしていってね！  

**霊夢:今回はコンペ理解のために、Miningについて検証していくぞ。**

**魔理沙:他に気になったことがあったら突っ込んでいくぞ。**

**Reimu: This time, I translated the code written by the organizer into Japanese to understand the competition.**

**Marisa: If you have any other concerns, I'll dig in.**

# Method

**霊夢:色々なagentを、特定のマップで動かしてみて、以下の4つの変動をグラフにして比較してみるよ。**

* Kore:Shipyardsが持っている資源の数
* Cargo:艦隊が持っている資源の数
* Ships:全体が持っている船の数
* All:これまでに獲得した資源の数 (Kore + Cargo + Ships×10 - 500)

**10,500 の値はそれぞれ、造船のコスト、ゲーム開始時に所持している資源の数の事です。**

**魔理沙:randomSeed = 42に設定して、Koreの配置は全検証において同じになるようにしたぜ**

**Reimu: Try running different agents on a specific map and compare the following four fluctuations in a graph.**

* Kore: Number of resources that Shipyards have
* Cargo: The number of resources the fleet has
* Ships: Number of ships the player has
* All: Number of resources acquired so far (Kore + Cargo + Ships × 10 -500)

**Values of 10,500 are the cost of shipbuilding and the number of resources you have at the start of the game, respectively.**

**Marisa: I set randomSeed = 42 so that Kore's placement is the same for all validations**

In [ ]:
%%capture
# install the latest version of kaggle_environments
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
import matplotlib.pylab as plt
env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)
print(env.name, env.version)

In [ ]:
def make_graph(steps):
    Kore = [steps[i][0]["reward"] for i in range(400)]
    Cargo = [0]*400
    Ships = [0]*400
    All = [0]*400
    for i in range(400):
        for x in steps[i][0]['observation']["players"][0][2].values():
            Cargo[i] += x[1]
    for i in range(400):
        for x in steps[i][0]['observation']["players"][0][1].values():
            Ships[i] += x[1]
        for x in steps[i][0]['observation']["players"][0][2].values():
            Ships[i] += x[2]
    for i in range(400):
        All[i] = Kore[i]+Cargo[i]+(Ships[i]-50)*10 
            
    fig, ax = plt.subplots(figsize = (20, 10))
    plt.rcParams["font.size"] = 12
    plt.subplot(2,2,1)
    plt.plot(Kore)
    plt.title("Kore")
    plt.grid()
    plt.subplot(2,2,2)    
    plt.plot(Cargo)
    plt.title("Cargo")
    plt.grid()
    plt.subplot(2,2,3)
    plt.plot(Ships)
    plt.title('Ships')
    plt.grid()
    plt.subplot(2,2,4)
    plt.plot(All)
    plt.title('All')
    plt.grid()
    # plt.title(df_main.loc[0,columns[0]])

In [ ]:
def vs_make_graph(step_list,agent_name):
    Kore = [] 
    Cargo = []
    Ships = []
    All = []
    for j,steps in enumerate(step_list):
        Kore.append([steps[i][0]["reward"] for i in range(400)])
        Cargo.append([0]*400) 
        Ships.append([0]*400)
        All.append([0]*400)
        for i in range(400):
            for x in steps[i][0]['observation']["players"][0][2].values():
                Cargo[j][i] += x[1]
        for i in range(400):
            for x in steps[i][0]['observation']["players"][0][1].values():
                Ships[j][i] += x[1]
            for x in steps[i][0]['observation']["players"][0][2].values():
                Ships[j][i] += x[2]
        for i in range(400):
            All[j][i] = Kore[j][i]+Cargo[j][i]+(Ships[j][i]-50)*10 

    fig, ax = plt.subplots(figsize = (20, 30))
    plt.rcParams["font.size"] = 20
    plt.subplot(4,1,1)
    for i in range(len(step_list)):    
        plt.plot(Kore[i],label=agent_name[i])
    plt.title("Kore")
    plt.legend()
    plt.grid()
    plt.subplot(4,1,2)    
    for i in range(len(step_list)):    
        plt.plot(Cargo[i],label=agent_name[i])
    plt.title("Cargo")
    plt.legend()
    plt.grid()
    plt.subplot(4,1,3)
    for i in range(len(step_list)): 
        plt.plot(Ships[i],label=agent_name[i])
    plt.title('Ships')
    plt.legend()
    plt.grid()
    plt.subplot(4,1,4)
    for i in range(len(step_list)):    
        plt.plot(All[i],label=agent_name[i])
    plt.title('All')
    plt.legend()
    plt.grid()

# No.1 one_box_mining

In [ ]:
%%writefile box_miner.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    period = 40
    
    for shipyard in me.shipyards:
        action = None
        if turn < 40:
            action = ShipyardAction.spawn_ships(1)
        elif turn % period == 1:
            action = ShipyardAction.launch_fleet_with_flight_plan(21, "E9N9W9S")
        elif turn % period == 3: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E8N")
        elif turn % period == 5: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E7N")
        elif turn % period == 7: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E6N")
        elif turn % period == 9: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E5N")
        elif turn % period == 11: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E4N")
        elif turn % period == 13: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3N")
        elif turn % period == 15: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2N")
        elif turn % period == 17: 
            action = ShipyardAction.launch_fleet_with_flight_plan(3, "E1N")
        elif turn % period == 19: 
            action = ShipyardAction.launch_fleet_with_flight_plan(2, "EN")
        elif turn % period == 21: 
            action = ShipyardAction.launch_fleet_with_flight_plan(2, "N")
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            
        
        shipyard.next_action = action

    return me.next_actions

**魔理沙:Kore Intro IIに出てきたbox_minerだぜ。**

**Marisa: It's the box_miner from Kore Intro II.**

In [ ]:
step1 = env.run(["/kaggle/working/box_miner.py"])
env.render(mode="ipython", width=1000, height=800)

## Results

In [ ]:
make_graph(step1)

# No.2 two_box_mining

In [ ]:
%%writefile two_minning.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def box_minning(agent,turn,box_size):

    period = 40
    for shipyard in agent.shipyards:
        if shipyard.ship_count >= 40 and turn % period == 1:
            box_size = 1
        if shipyard.ship_count >= 89 and turn % period == 1:
            box_size = 2
        action = None
        
        
        if turn > 40:
            if turn % period == 1:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "E9N9W9S")
            elif turn % period == 3: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E8N")
            elif turn % period == 5: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E7N")
            elif turn % period == 7: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E6N")
            elif turn % period == 9: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E5N")
            elif turn % period == 11: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E4N")
            elif turn % period == 13: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3N")
            elif turn % period == 15: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2N")
            elif turn % period == 17: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E1N")
            elif turn % period == 19: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "EN")
            elif turn % period == 21: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "N")
        if box_size > 1:
            if turn % period == 2:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "W9S9E9N")
            elif turn % period == 4: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W8S")
            elif turn % period == 6: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W7S")
            elif turn % period == 8: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W6S")
            elif turn % period == 10: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W5S")
            elif turn % period == 12: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W4S")
            elif turn % period == 14: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W3S")
            elif turn % period == 16: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W2S")
            elif turn % period == 18: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W1S")
            elif turn % period == 20: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "WS")
            elif turn % period == 22: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "S")
            
        shipyard.next_action = action
        
    return int(box_size)

In [ ]:
%%writefile spawn.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def spawn(agent,spawn_cost,kore_left):
    for shipyard in agent.shipyards:
        if shipyard.next_action:
            continue
        if kore_left >= spawn_cost * shipyard.max_spawn:
            action = ShipyardAction.spawn_ships(shipyard.max_spawn)
            shipyard.next_action = action
            kore_left -= spawn_cost * shipyard.max_spawn
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action
            kore_left -= spawn_cost

    return

In [ ]:
%%writefile main.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from two_minning import box_minning
from spawn import spawn

box_num = 0

def agent(obs, config):
    global box_num
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    
    box_num = box_minning(me,turn,box_num)
    spawn(me,spawn_cost,kore_left)
    return me.next_actions

In [ ]:
step2 = env.run(["/kaggle/working/main.py"])
env.render(mode="ipython", width=1000, height=800)

## Results

In [ ]:
make_graph(step2)

# No.3 four_box_miner

In [ ]:
%%writefile four_minning.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def box_minning(agent,turn,box_size):

    period = 40
    for shipyard in agent.shipyards:
        if shipyard.ship_count >= 40 and turn % period == 10 and box_size < 1:
            box_size = 1
        if shipyard.ship_count >= 89 and turn % period == 10 and box_size < 2:
            box_size = 2
        if shipyard.ship_count >= 140 and turn % period == 10 and box_size < 3:
            box_size = 3
        if shipyard.ship_count >= 190 and turn % period == 10 and box_size < 4:
            box_size = 4
        action = None
        
        
        if turn > 9:
            if turn % period == 10:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "E9N9W9S")
            elif turn % period == 12: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E8N")
            elif turn % period == 14: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E7N")
            elif turn % period == 16: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E6N")
            elif turn % period == 18: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E5N")
            elif turn % period == 20: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E4N")
            elif turn % period == 22: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3N")
            elif turn % period == 24: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2N")
            elif turn % period == 26: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E1N")
            elif turn % period == 28: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "EN")
        if box_size > 1:
            if turn % period == 11:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "W9S9E9N")
            elif turn % period == 13: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W8S")
            elif turn % period == 15: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W7S")
            elif turn % period == 17: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W6S")
            elif turn % period == 19: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W5S")
            elif turn % period == 21: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W4S")
            elif turn % period == 23: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W3S")
            elif turn % period == 25: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W2S")
            elif turn % period == 27: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W1S")
            elif turn % period == 29: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "WS")
        if box_size > 2:
            if turn % period == 30:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "E9S9W9N")
            elif turn % period == 32: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E8S")
            elif turn % period == 34: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E7S")
            elif turn % period == 36: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E6S")
            elif turn % period == 38: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E5S")
            elif turn % period == 0: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E4S")
            elif turn % period == 2: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E3S")
            elif turn % period == 4: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E2S")
            elif turn % period == 6: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "E1S")
            elif turn % period == 8: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "ES")
        if box_size > 3:
            if turn % period == 31:
                action = ShipyardAction.launch_fleet_with_flight_plan(21, "W9N9E9S")
            elif turn % period == 33: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W8N")
            elif turn % period == 35: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W7N")
            elif turn % period == 37: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W6N")
            elif turn % period == 39: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W5N")
            elif turn % period == 1: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W4N")
            elif turn % period == 3: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W3N")
            elif turn % period == 5: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W2N")
            elif turn % period == 7: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3, "W1N")
            elif turn % period == 9: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2, "WN")    
        shipyard.next_action = action
        
    return int(box_size)

In [ ]:
%%writefile main2.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from four_minning import box_minning
from spawn import spawn

box_num = 0

def agent(obs, config):
    global box_num
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    
    box_num = box_minning(me,turn,box_num)
    spawn(me,spawn_cost,kore_left)
    return me.next_actions

In [ ]:
step3 = env.run(["/kaggle/working/main2.py"])
env.render(mode="ipython", width=1000, height=800)

## Results

In [ ]:
make_graph(step3)

# one_box_mining vs two_box_mining vs four_box_mining

In [ ]:
step_list = [step1,step2,step3]
agent_name = ["one_box_mining","two_box_mining","four_box_mining"]
vs_make_graph(step_list,agent_name)

**魔理沙:Koreは、200ターン目までどのエージェントも同じくらいだけど、最終的には、four_box_miningが一番多くなってるぜ。**

**霊夢:Cargoは150ターン目から増加してきて、300ターン目あたりからは振幅がほぼ一定になるね。**

**魔理沙:Shipsは200ターン目あたりまではどのエージェントも同じくらいだけど、そこからone_box_miningは少しずつ増加していって、two_box_miningは大きく増加していって、four_box_miningは完全に増加しなくなるね。four_box_miningが増加しなくなるのは、艦隊を出し続けているせいで、造船する暇がないからだろうな**

**霊夢:All (取得した資源の総数：Kore + Cargo + Ships*10 -500) は、150ターン目あたりまではどのエージェントも大差ないけど、そこから差が開いて行って、最終的にはfour_box_miningが最も多くなったよ。**

**魔理沙:two_box_miningはone_box_miningの約2倍になってるけど、for_box_miningはtwo_box_miningの2倍やone_box_miningの4倍より少なそうだね。**

**Marisa: Kore is about the same for all agents until turn 200, but in the end, four_box_mining is the most.**

**Reimu: Cargo increases from turn 150, and the amplitude becomes almost constant from around turn 300.**

**Marisa: Ships is about the same for all agents until around turn 200, but from there one_box_mining is increasing little by little, two_box_mining is increasing significantly, and four_box_mining isn't increasing altogether. The reason why four_box_mining doesn't increase is probably because we don't have time to build a ship because we keep out the fleet**

**Reimu: All (total number of resources acquired: Kore + Cargo + Ships * 10 -500) is not much different for any agent until around the 150th turn, but the difference comes out from there, and finally four_box_mining is the most.**

**Marisa: Two_box_mining is about twice as much as one_box_mining, but for_box_mining is less than twice as much as two_box_mining and four times as much as one_box_mining.**

# No.4 4box&1shipyard

In [ ]:
%%writefile four_minning2.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def box_minning(agent,turn,box_size,ships_num):

    period = 40
    for shipyard in agent.shipyards:
        if not shipyard.id == '0-1':
            continue
        if not shipyard.next_action == None:
            continue
        
        if shipyard.ship_count >= 40 and turn % period == 10 and box_size < 1:
            box_size = 1
        if shipyard.ship_count >= 89 and turn % period == 10 and box_size < 2:
            box_size = 2
        if shipyard.ship_count >= 140 and turn % period == 10 and box_size < 3:
            box_size = 3
        if shipyard.ship_count >= 190 and turn % period == 10 and box_size < 4:
            box_size = 4
        action = None
        
        k =0 if box_size == 0 else (ships_num - 47*box_size) // (10 * box_size)
        
        if turn > 9:
            if turn % period == 10:
                action = ShipyardAction.launch_fleet_with_flight_plan(21 + k, "E9N9W9S")
            elif turn % period == 12: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E8N")
            elif turn % period == 14: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E7N")
            elif turn % period == 16: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E6N")
            elif turn % period == 18: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E5N")
            elif turn % period == 20: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E4N")
            elif turn % period == 22: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E3N")
            elif turn % period == 24: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E2N")
            elif turn % period == 26: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E1N")
            elif turn % period == 28: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2 + k, "EN")
        if box_size > 1:
            if turn % period == 11:
                action = ShipyardAction.launch_fleet_with_flight_plan(21 + k, "W9S9E9N")
            elif turn % period == 13: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W8S")
            elif turn % period == 15: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W7S")
            elif turn % period == 17: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W6S")
            elif turn % period == 19: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W5S")
            elif turn % period == 21: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W4S")
            elif turn % period == 23: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W3S")
            elif turn % period == 25: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W2S")
            elif turn % period == 27: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W1S")
            elif turn % period == 29: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2 + k, "WS")
        if box_size > 2:
            if turn % period == 30:
                action = ShipyardAction.launch_fleet_with_flight_plan(21 + k, "E9S9W9N")
            elif turn % period == 32: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E8S")
            elif turn % period == 34: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E7S")
            elif turn % period == 36: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E6S")
            elif turn % period == 38: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E5S")
            elif turn % period == 0: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E4S")
            elif turn % period == 2: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E3S")
            elif turn % period == 4: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E2S")
            elif turn % period == 6: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "E1S")
            elif turn % period == 8: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2 + k, "ES")
        if box_size > 3:
            if turn % period == 31:
                action = ShipyardAction.launch_fleet_with_flight_plan(21 + k, "W9N9E9S")
            elif turn % period == 33: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W8N")
            elif turn % period == 35: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W7N")
            elif turn % period == 37: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W6N")
            elif turn % period == 39: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W5N")
            elif turn % period == 1: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W4N")
            elif turn % period == 3: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W3N")
            elif turn % period == 5: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W2N")
            elif turn % period == 7: 
                action = ShipyardAction.launch_fleet_with_flight_plan(3 + k, "W1N")
            elif turn % period == 9: 
                action = ShipyardAction.launch_fleet_with_flight_plan(2 + k, "WN")    
        shipyard.next_action = action
        
    return int(box_size)

%%writefile make_shipyard.py

In [ ]:
%%writefile sub_function.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

def make_shipyard(agent,turn,box_num):
    period = 40
    action = None
    for shipyard in agent.shipyards:
        if not shipyard.id == '0-1':
            continue
        if box_num == 2 and (turn % period > 29 or turn % period < 10) and shipyard.ship_count >= 50:
            action = ShipyardAction.launch_fleet_with_flight_plan(50, "NC")
            shipyard.next_action = action
            return True
        
    return False

def deport(agent,turn):
    period = 40
    for shipyard in agent.shipyards:
        action = None
        if shipyard.id == '0-1':
            continue
        if shipyard.ship_count >= 47:
            action = ShipyardAction.launch_fleet_with_flight_plan(shipyard.ship_count, "S")
        shipyard.next_action = action
    
    return
            
def all_ship_count(agent):
    res = 0
    for shipyard in agent.shipyards:
        res += shipyard.ship_count
    
    for fleet in agent.fleets:
        res += fleet.ship_count
    return res

In [ ]:
%%writefile main3.py
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
from four_minning2 import box_minning
from sub_function import make_shipyard,deport,all_ship_count
from spawn import spawn

box_num = 0
make = False
def agent(obs, config):
    global box_num
    global make
    board = Board(obs, config)
    me=board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore
    ships = all_ship_count(me)
    
    if not make: make = make_shipyard(me,turn,box_num)    
    box_num = box_minning(me,turn,box_num,ships)
    deport(me,turn)
    spawn(me,spawn_cost,kore_left)
    return me.next_actions

In [ ]:
step4 = env.run(["/kaggle/working/main3.py"])
env.render(mode="ipython", width=1000, height=800)

In [ ]:
make_graph(step4)

# four_box_minnig vs 4box&1shipyard

In [ ]:
step_list = [step3,step4]
agent_name = ["four_box_mining","4box&1shipyard"]
vs_make_graph(step_list,agent_name)

In [ ]:
!mkdir ./submission 
!cp ./four_minning2.py ./submission/four_minning2.py
!cp ./sub_function.py ./submission/sub_function.py
!cp ./spawn.py ./submission/spawn.py
!cp ./main3.py ./submission/main.py
!zip -r ./submission.tar.gz ./submission 